## Problem Set 3. Makarova Anastasia.

###  Problem 1.



As the Skolkovo campus is being built, there is a need to level the hill (the elevation profile is shown below on the left) to obtain a flat surface with the elevation profile shown below on the right. Assume that the shape of each tile is square and that the cost of moving a certain amount of earth between the tiles is proportional to the Euclidean distance between the tile centers. Formulate the problem of optimal leveling strategy (determining how to move earth) as a network flow program, and then solve it using a generic LP solver (CVX). Check whether the optimal strategy you obtain is integer.

#### Solution

* n = 20 number of segments
* Matrix  $ C_{n\times n}$, $c_{ij}$ - cost of transportation from segment i to j = euclidian distance between i and j.
* Matrix $E_{n\times n}$, $e_{ij}$ - earth add/movement from i to j 
* D - donors, R- recipients
* Vector $ d_{1\times n}$, $d_i$ - how much we should remove to make it equal to mean hight
* Then, the problem can be formulated as
\begin{equation*}
\begin{aligned}
& \underset{x}{\text{minimize}}
& & \sum_{i,j} c_{ij} e_{ij} \\
& \text{subject to}
& & d_j + \sum_{i \in R(j)} e_{ij} = \sum_{i \in D(j)} e_{ji}, \\
&&& e_{ij} \geq 0, e_{ij} \leq d_i.
\end{aligned}
\end{equation*}

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
from numpy.matlib import repmat
import cvxpy as cvx
import gurobipy as grb

np.random.seed(1)

C = 20 # number of clients
clients = np.random.rand(2,C) #client positions
F = 15 #number of facilities
facilities = np.random.rand(2,F)

capacities = np.ones((F,), dtype=np.int)*4 #maximum number of clients per facility

dx = repmat(clients[0,:],F,1) - repmat(facilities[0,:],C,1).transpose()
dy = repmat(clients[1,:],F,1) - repmat(facilities[1,:],C,1).transpose()

assignment_costs = np.zeros((F, C))
assignment_costs = 3*(dx*dx + dy*dy) #the assignment cost is the distance squared

opening_costs = np.ones((F,))

In [3]:
np.random.seed(1)
m = 4
n = 5

heights = np.array([[5, 5, 10, 10, 10],
[5, 5, 10, 20, 10],
[0, 5, 5, 10, 5],
[0, 0, 0, 5, 0]],dtype = np.int)

mean = np.mean(heights, dtype = np.int)
donors = []
recipients = []

""" Find donors and recipients """
for i in range(heights.shape[0]):
    for j in range(heights.shape[1]):
        if heights[i, j] > mean:
            donors.append((heights[i, j] - mean, i, j))
        else:
            recipients.append((heights[i, j] - mean, i, j))


"""Distances between donors and recepients"""
dist = np.zeros((len(donors), len(recipients)))
earth_d = np.zeros(len(donors))
earth_r = np.zeros(len(recipients))

for j in range(len(recipients)):
        earth_r[j] = - recipients[j][0]

for i in range(len(donors)):
    earth_d[i] = donors[i][0]
    for j in range(len(recipients)):
        dist[i, j] = np.linalg.norm(np.array((donors[i][1], donors[i][2]))
                                    -  np.array((recipients[j][1], recipients[j][2])))
        

f = cvx.Variable(len(donors), len(recipients))

constraints = [f >= 0]

for i in range(len(donors)):
    constraints.append(cvx.sum_entries(f[i:(i + 1), :]) == earth_d[i])
    
for i in range(len(recipients)):
    constraints.append(cvx.sum_entries(f[:, i:(i + 1)]) == earth_r[i])
    
for i in range(len(donors)):
    for j in range(len(recipients)):
        constraints.append(f[i, j] <= earth_d[i])

# Form objective.
objective = cvx.Minimize(cvx.trace(f * dist.T))

# Form and solve problem.
solution = cvx.Problem(objective, constraints)
solution.solve(solver = 'GUROBI')

print('Optimal value of obj = {0}'.format(solution.value))
print('Matrix of earth movements \n {0}'.format(f.value))


Optimal value of obj = 95.4910638367
Matrix of earth movements 
 [[ 1.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  4.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  3.]
 [ 0.  0.  0.  0.  2.  1.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  5.  6.  2.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  3.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  4.  0.  0.]]


### Problem 2
Implement a branch-and-bound solver for the capacitated facility location problem you were facing in the first assignment. Be careful to branch on the right variables.

#### Solution

* F facilities, C clients
* Matrix $X_{F \times C}$: $x_{ij} \in \{0,1\} $ whether client $j$ belonges to facility $i$.
* Vector $y_{F \times 1}$ $y_i \in \{0,1\}$ whether facility $i$ is opened


\begin{equation*}
\begin{aligned}
& \underset{x, y}{\text{minimize}}
& & \sum_{i=1}^F \sum_{j=1}^C x_{ij} k_{ij} + \sum_{i=1}^F y_i c_i \\
& \text{subject to}
& & \sum_{i=1}^F x_{ij} = 1,~\forall j, \\
&&& \sum_{j=1}^C x_{ij} \leq cap_i y_i,~\forall i, \\
&&& cap - capacities of facilities.
\end{aligned}
\end{equation*}

* Firstly, let's remember the gurobi solution and then compare with the branch and bound method (original and recursive). Recursive method is not so optimal in terms of memory, but more intuitive. 

* It can be seen from the last experiment, that without bound and branch vector $y$ is noninteger. So at it step of branching we choose the closest to 0.5 component of the vector y, recreate constraints and solve new LP problem. 

* The result for branch and bound with MAX_ITER = 200 is the same as Gurobi model result (Best objective 7.571112923135e+00): original 7.57111292313 and recursive 7.66898384142.

In [13]:
np.random.seed(1)

C = 20 # number of clients
clients = np.random.rand(2,C) #client positions
F = 15 #number of facilities
facilities = np.random.rand(2,F)

capacities = np.ones((F,), dtype=np.int)*4 #maximum number of clients per facility

dx = repmat(clients[0,:],F,1) - repmat(facilities[0,:],C,1).transpose()
dy = repmat(clients[1,:],F,1) - repmat(facilities[1,:],C,1).transpose()

assignment_costs = np.zeros((F, C))
assignment_costs = 3*(dx*dx + dy*dy) #the assignment cost is the distance squared

opening_costs = np.ones((F,))

In [35]:
"""Gurobi Solution"""
from gurobipy import *
# create the model
m = Model("facility")

y = []
for i_f in range(F):
    y.append(m.addVar(vtype=GRB.BINARY))

x = []    
for i_f in range(F):
    x.append([])
    for i_c in range(C):
        x[i_f].append(m.addVar(vtype=GRB.BINARY))

# the objective is to minimize the total fixed and variable costs
m.modelSense = GRB.MINIMIZE

# update model to integrate new variables
m.update()

# set optimization objective - minimize sum of fixed costs
obj_summands = []
for i_f in range(F):
    obj_summands.append(opening_costs[i_f]*y[i_f])
    
for i_f in range(F):
    for i_c in range(C):
        obj_summands.append(assignment_costs[i_f][i_c]*x[i_f][i_c])

m.setObjective(quicksum(obj_summands))  

# set constraints
for i_c in range(C):
    client_constr_summands = [x[i_f][i_c] for i_f in range(F)]
    m.addConstr(sum(client_constr_summands), GRB.EQUAL, 1.0)
        
for i_f in range(F):        
    facility_constr_summands = [x[i_f][i_c] for i_c in range(C)]
    m.addConstr(sum(facility_constr_summands), GRB.LESS_EQUAL, capacities[i_f]*y[i_f])       

for i_f in range(F):        
    facility_constr_summands = [x[i_f][i_c] for i_c in range(C)]
    m.addConstr(max(facility_constr_summands), GRB.LESS_EQUAL, y[i_f])     

# optimize
  
m.optimize()

Optimize a model with 50 rows, 315 columns and 645 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 4e+00]
  Objective range [2e-04, 5e+00]
  Bounds range    [1e+00, 1e+00]
  RHS range       [1e+00, 1e+00]
Found heuristic solution: objective 33.3461
Presolve time: 0.03s
Presolved: 50 rows, 315 columns, 645 nonzeros
Variable types: 0 continuous, 315 integer (315 binary)

Root relaxation: objective 6.504420e+00, 63 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    6.50442    0    9   33.34610    6.50442  80.5%     -    0s
H    0     0                      11.5023502    6.50442  43.5%     -    0s
H    0     0                      10.5750192    6.50442  38.5%     -    0s
H    0     0                       8.1675755    6.50442  20.4%     -    0s
H    0     0                       7.9698679    6.50442  18.4%     -    0s
     0     0    6.865

In [64]:
class facility:
    
    def __init__(self):
        
        self.x = cvx.Variable(F, C)
        self.y = cvx.Variable(F)
        #self.constraints = []
        self.obj = None
        self.MAX_ITER = 200
        self.iter_count = 0
        self.best_obj = np.inf
        self.opened_facilities = []
        
    def make_constraints(self): 
        """ Make constraints for the main node"""
        
        constraints = [self.x >= 0, self.x <= 1,
                            self.y >= 0, self.y <= 1]
        
        #for i in range(C):
        #    constraints.append(cvx.sum_entries(self.x[:, i: i + 1] == 1))
        #for j in range(F):
         #   constraints.append(self.x[j:j + 1, :] <= capacities[j]*self.y[j])
        
        IF = np.identity(F)
        IC = np.identity(C) 
        for i_c in xrange(C):
            constraints.append(np.ones(F)*self.x*IC[i_c] == 1)
        for i_f in xrange(F):
            constraints.append(IF[i_f]*self.x*np.ones(C) <= capacities[i_f]*self.y[i_f])

        self.obj = cvx.Minimize(cvx.trace(self.x*assignment_costs.T) + opening_costs*self.y)
        
        return constraints 

    def find_solution(self, constraints):
        """ Solve with the current constraints"""
        
        prob = cvx.Problem(self.obj, constraints)
        sol = prob.solve(solver = 'GUROBI')
        return sol
    
    def pick_facility(self):
        """ Find the most suitable facility to branch"""
        
        err = np.inf
        fclt = 0
        for (i,var) in enumerate(self.y.value):
            new_err = abs(var[0, 0] - 0.5)
            if (new_err <= err):
                fclt = i
                err = new_err
        if (err > 0.49):
            return -1
        else:
            return fclt
                                    
    def branch_and_bound(self, constraints):
    
        L = []
        self.find_solution(constraints)
        for i in range(self.MAX_ITER):
        
            fclt = self.pick_facility()
            for j in range(2):
                child_constraints = constraints + [self.y[fclt] == j]
                objective = self.find_solution(constraints)
                
                if objective >= self.best_obj:
                    continue
                               
                fclt = self.pick_facility()
                if fclt == -1:
                    self.best_obj = objective
                    self.opened_facilities = self.y.value
                    
                else:
                    L.append(child_constraints)
                    
            if (len(L) == 0):
                return self.best_obj
            
            constraints = L.pop()
            objective = self.find_solution(constraints)
            
    def branch_and_bound_rec(self, constraints):
            
        if self.iter_count > self.MAX_ITER:
            return 

        objective = self.find_solution(constraints)
        if objective >= self.best_obj:
            return

        fclt = self.pick_facility()
        if fclt == -1 and objective < self.best_obj:
            self.best_obj = objective
            self.opened_facilities = self.y.value
            return
        
        self.iter_count += 2
        for j in range(2):
          self.branch_and_bound_rec(constraints + [self.y[fclt] == j])

                               
                               
                               

In [65]:
"""Non recursive"""

fl = facility()
constraints = fl.make_constraints()
fl.branch_and_bound(constraints)

print('Best objective: \n{0}'.format(fl.best_obj))
print('Opened facilities: \n{0}'.format(fl.opened_facilities))

Best objective: 
7.57111292313
Opened facilities: 
[[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 0.]]


In [66]:
"""Recursive"""

fl = facility()
constraints = fl.make_constraints()
fl.branch_and_bound_rec(constraints)

print('Best objective rec: \n{0}'.format(fl.best_obj))
print('Opened facilities: \n{0}'.format(fl.opened_facilities))

Best objective rec: 
7.66898384142
Opened facilities: 
[[ 0.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 0.]]


In [68]:
"""Solution without bound and branch """

fl = facility()
constraints = fl.make_constraints()
sol = fl.find_solution(constraints)

print('Best objective rec: \n{0}'.format(sol))
print('Opened facilities: \n{0}'.format(fl.y.value))


Best objective rec: 
6.50235022407
Opened facilities: 
[[ 0.  ]
 [ 0.25]
 [ 0.  ]
 [ 0.25]
 [ 0.5 ]
 [ 0.75]
 [ 0.25]
 [ 0.75]
 [ 0.  ]
 [ 0.5 ]
 [ 0.25]
 [ 0.75]
 [ 0.  ]
 [ 0.75]
 [ 0.  ]]


### Problem 3


A group of 20 students are deciding how to fill the 10 room dormitory (the rooms are identical and each room hosts two students). Each pair of students has a certain preference on how much they would like to live together (generate a random symmetric matrix for that). You therefore want to split students into pairs in order to maximize the total preference.
Formulate this problem as an ILP and solve it using an ILP solver (Gurobi/Mosek, etc.)
Consider the LP relaxation, and visualize the solution. This visualization should suggest you the cuts that can tighten your relaxation. Implement the procedure that would find such cuts (the separation oracle) and run the cutting plane algorithm. Verify that you are able to get a fully integer solution when enough cuts are added into the program.
Evaluate the performance of the generic and your own ILP solvers for larger groups of students (how well do they scale?) Consider random uniformly [0;1]-distributed matrices vs. random uniformly distributed binary matrices (student preferences are like/dislike i.e. 0 or 1).

#### Solution

* $n = 20$ - number of students
* Matrix $p$ $n \times n$ - matrix of students preferencies
* Symmetric matrix $z$ $ n \times n $ - matrix of {0,1}, where $z_{ji} = z_{ij}$ means if student $j$ live with student $i$
* Then, LP problem formulation

$$\max \sum p_{ij}z_{ij}  $$
$$s.t. \ \  \forall j \ \ \sum_{i}^{n} z_{ij} = 1$$
$$ \forall i,j \ \ z_{ji} = z_{ij}$$


In [8]:
np.random.seed(1)

N = 20 # number of students

preferences = np.random.random((N, N))
preferences = (preferences + preferences.T)/2


In [9]:
"""Gurobi model"""
m = grb.Model("dormitory")

z = []
for i in range(N):
    z.append([])
    for j in range(N):
        z[i].append(m.addVar(vtype=grb.GRB.BINARY))
        
# the objective is to maximize the total preferences
m.modelSense = grb.GRB.MAXIMIZE

# update model to integrate new variables
m.update()

# set optimization objective - sum of all preferences
obj= []
for i in range(N):
    for j in range(N):
        obj.append(preferences[i][j]*z[i][j])
m.setObjective(grb.quicksum(obj)) 

# set constraints
for i in range(N):
    constr = [z[i][j] for j in range(N)]
    m.addConstr(sum(constr), grb.GRB.EQUAL, 1.0)
    
for i in range(N):
    for j in range(i + 1, N):
        m.addConstr(z[i][j] - z[j][i], grb.GRB.EQUAL, 0.0)

m.optimize()

Optimize a model with 210 rows, 400 columns and 780 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [6e-02, 1e+00]
  Bounds range    [1e+00, 1e+00]
  RHS range       [1e+00, 1e+00]
Found heuristic solution: objective 10.3629
Presolve removed 190 rows and 190 columns
Presolve time: 0.12s
Presolved: 20 rows, 210 columns, 400 nonzeros
Variable types: 0 continuous, 210 integer (210 binary)

Root relaxation: objective 1.718242e+01, 41 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      17.1824249   17.18242  0.00%     -    0s

Explored 0 nodes (41 simplex iterations) in 0.30 seconds
Thread count was 4 (of 4 available processors)

Optimal solution found (tolerance 1.00e-04)
Best objective 1.718242487604e+01, best bound 1.718242487604e+01, gap 0.0%


In [10]:
C = cvx.Variable(N, N)

constraints = []

for i in range(N):
    constraints.append(cvx.sum_entries(C[i:i + 1, :]) == 1)
    constraints.append(cvx.sum_entries(C[:, i:i + 1]) == 1)
    for j in range(N):
        constraints.append(C[i, j] >= 0)
        constraints.append(C[i, j] <= 1)
        constraints.append(C[i, j] <= cvx.sum_entries(C[:, j:j+1]))
        constraints.append(C[i, j] <= cvx.sum_entries(C[i:i+1, :]))

objective = cvx.Maximize(cvx.trace(C * preferences.T))

solution = cvx.Problem(objective, constraints)
solution.solve(solver = 'GUROBI')

print('Optimal value of objective = {0}'.format(solution.value))
print('Resettlement: \n{0}'.format(C.value))

Optimal value of objective = 17.182424876
Resettlement: 
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.
   0.  0.]
 [ 0.  0